In [16]:
# Import the data from csv file and explore the data

import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import json

# **** SETUP ****

# change to match your filesystem
DATA_DIR = "./data/"
DEFAULT_RECEIPTS_FILE = os.path.join(DATA_DIR, "World_Energy_Consumption.csv")
PROJECT_NAME = "deb-01-371820"
DATASET_NAME = "emissions"

data = []

df = pd.read_csv(DEFAULT_RECEIPTS_FILE)

display(df.head(n=10))



,iso_code,country,year,coal_prod_change_pct,coal_prod_change_twh,gas_prod_change_pct,gas_prod_change_twh,oil_prod_change_pct,oil_prod_change_twh,energy_cons_change_pct,...,solar_elec_per_capita,solar_energy_per_capita,gdp,wind_share_elec,wind_cons_change_pct,wind_share_energy,wind_cons_change_twh,wind_consumption,wind_elec_per_capita,wind_energy_per_capita
0,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,1901,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,1902,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,1903,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,1904,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AFG,Afghanistan,1905,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AFG,Afghanistan,1906,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AFG,Afghanistan,1907,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AFG,Afghanistan,1908,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AFG,Afghanistan,1909,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# create the gdf dataframe in order to prep for fct_gdp table creation


# extract necessary columns from raw csv file
gdp_df = df
gdp_cols = ['iso_code', 'country', 'year', 'gdp', 'population']
gdp_df = gdp_df[gdp_cols]


# rename columns to standardize across all the tables
gdp_df = gdp_df.rename(columns={
    'iso_code': 'country_code', 
    'country':'country_name', 
    })

# create composite key column for the fct_gdp table
def composite_key(row):
    comp_key = f"{row.country_code}{row.year}"
    return comp_key

gdp_df.insert(0, 'country_code_year', gdp_df.apply(composite_key, axis=1))

gdp_df.drop_duplicates(inplace=True)

# create created_at and modified_at columns
gdp_df['created_at'] = pd.Timestamp.now()
gdp_df['modified_at'] = None

gdp_df


display(gdp_df)




,country_code_year,country_code,country_name,year,gdp,population,created_at,modified_at
0,AFG1900,AFG,Afghanistan,1900,NaN,5021241.0,2023-01-17 15:43:46.331285,None
1,AFG1901,AFG,Afghanistan,1901,NaN,5053439.0,2023-01-17 15:43:46.331285,None
2,AFG1902,AFG,Afghanistan,1902,NaN,5085403.0,2023-01-17 15:43:46.331285,None
3,AFG1903,AFG,Afghanistan,1903,NaN,5118005.0,2023-01-17 15:43:46.331285,None
4,AFG1904,AFG,Afghanistan,1904,NaN,5150814.0,2023-01-17 15:43:46.331285,None
...,...,...,...,...,...,...,...,...
17427,ZWE2015,ZWE,Zimbabwe,2015,2.503057e+10,13815000.0,2023-01-17 15:43:46.331285,None
17428,ZWE2016,ZWE,Zimbabwe,2016,2.515176e+10,14030000.0,2023-01-17 15:43:46.331285,None
17429,ZWE2017,ZWE,Zimbabwe,2017,NaN,14237000.0,2023-01-17 15:43:46.331285,None
17430,ZWE2018,ZWE,Zimbabwe,2018,NaN,14439000.0,2023-01-17 15:43:46.331285,None


In [28]:
consump_df = df
consump_cols = [
    'iso_code', 
    'country', 
    'year', 
    'biofuel_consumption', 
    'coal_consumption',
    'fossil_fuel_consumption',
    'gas_consumption',
    'hydro_consumption',
    'low_carbon_consumption',
    'nuclear_consumption',
    'oil_consumption',
    'other_renewable_consumption',
    'primary_energy_consumption',
    'renewables_consumption',
    'solar_consumption',
    'wind_consumption']

consump_df = consump_df[consump_cols]

consump_df = consump_df.rename(columns={
    'iso_code': 'country_code', 
    'country':'country_name', 
    })

consump_df.drop_duplicates(inplace=True)

# create composite key column for the fct_gdp table
def composite_key(row):
    comp_key = f"{row.country_code}{row.year}"
    return comp_key

consump_df.insert(0, 'country_code_year', consump_df.apply(composite_key, axis=1))

# create created_at and modified_at columns
consump_df['created_at'] = pd.Timestamp.now()
consump_df['modified_at'] = None

display(consump_df)

,country_code_year,country_code,country_name,year,biofuel_consumption,coal_consumption,fossil_fuel_consumption,gas_consumption,hydro_consumption,low_carbon_consumption,nuclear_consumption,oil_consumption,other_renewable_consumption,primary_energy_consumption,renewables_consumption,solar_consumption,wind_consumption,created_at,modified_at
0,AFG1900,AFG,Afghanistan,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
1,AFG1901,AFG,Afghanistan,1901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
2,AFG1902,AFG,Afghanistan,1902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
3,AFG1903,AFG,Afghanistan,1903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
4,AFG1904,AFG,Afghanistan,1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17427,ZWE2015,ZWE,Zimbabwe,2015,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.642,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
17428,ZWE2016,ZWE,Zimbabwe,2016,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.500,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
17429,ZWE2017,ZWE,Zimbabwe,2017,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
17430,ZWE2018,ZWE,Zimbabwe,2018,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-17 15:46:29.394667,None
